In [3]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
#from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
#from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

**Notes:** 

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

The notebook exploits [pyVO](https://pyvo.readthedocs.io/en/latest/), an advanced library  of The [Virtual Observatory](https://ivoa.net/).

[Astroquery](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html) (not used here) is a well-documented, user-friendly alternative.

--------------------------------------------------------

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1
**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [4]:
# the catalogue name in VizieR
CATALOGUE = "J/MNRAS/505/5978"

We first retrieve the catalogue information.

In [5]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [6]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

Gaia EDR3 view on Galactic globular clusters
Short Name: J/MNRAS/505/5978
IVOA Identifier: ivo://cds.vizier/j/mnras/505/5978
Access modes: conesearch, tap#aux, web
Multi-capability service -- use get_service()

We use the data from Gaia Early Data Release 3 (EDR3) to study the kinematic
properties of Milky Way globular clusters. We measure the mean parallaxes and
proper motions (PM) for 170 clusters, determine the PM dispersion profiles for
more than 100 clusters, uncover rotation signatures in more than 20 objects,
and find evidence for radial or tangential PM anisotropy in a dozen richest
clusters. At the same time, we use the selection of cluster members to explore
the reliability and limitations of the Gaia catalogue itself. We find that the
formal uncertainties on parallax and PM are underestimated by 10-20% in dense
central regions even for stars that pass numerous quality filters. We explore
the spatial covariance function of systematic errors, and determine a lower
limit on the

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [ ]:
voresource.creators[0]

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [12]:
tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")

In this catalogue, we have 1 tables.
J/MNRAS/505/5978/tablea1: The catalogue of cluster proper motions


In [9]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

['J/MNRAS/505/5978/tablea1']

In [11]:
print("read", tables.items())

read dict_items([('J/MNRAS/505/5978/tablea1', <VODataServiceTable name="J/MNRAS/505/5978/tablea1">... 15 columns ...</VODataServiceTable>)])


The actual data can then be accessed using any of the ``access_modes`` of the voresource.

In [ ]:
voresource.access_modes()

The web access is found by following the ``reference_url``

In [ ]:
voresource.reference_url

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [13]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records

<DALResultsTable length=10>
recno    Name   OName  RAJ2000 ...  Rscale Nstar         SimbadName        
                         deg   ...  arcmin                                 
int32   object  object float64 ... float64 int32           object          
----- --------- ------ ------- ... ------- ----- --------------------------
    1   NGC 104 47 Tuc   6.024 ...   10.03 39932                    NGC 104
    2   NGC 288         13.188 ...    2.79  4689                    NGC 288
    3   NGC 362         15.809 ...    3.37  3878                    NGC 362
    4 Whiting 1         30.737 ...    0.52    40                  Whiting 1
    5  NGC 1261         48.068 ...    1.73  1218                   NGC 1261
    6     Pal 1         53.333 ...    0.54    92                      Pal 1
    7       E 1   AM 1   58.76 ...    0.36    58                        E 1
    8  Eridanus         66.185 ...    0.52    44 NAME Eridanus Star Cluster
    9     Pal 2         71.525 ...    0.64   180            

<!-- section position -->

We will explore the other access modes here.

### 3.2. Execute a cone search query

We use the Simple Cone Search (<a href='https://www.ivoa.net/documents/latest/ConeSearch.html'>SCS</a>) protocol of the virtual observatory (only available for tables with positions).

Finding the conesearch service that you want to use (there is usually one per table):

Let's use the first one for this example. 

In [ ]:
# we get the conesearch  service associated to the first table
conesearch_interface = voresource.get_interface(service_type='conesearch', 
                                                keyword='J/MNRAS/505/5978/tablea1',
                                                lax=True)
# if you get a TypeError about an unexpected keyword, check that you installed pyvo>=1.6
conesearch_service = conesearch_interface.to_service()

We adapt the radius and position of the center to our needs:

In [ ]:
conesearch_radius = 1 / 60.0  # in degrees
conesearch_center = (71.520996, 31.388166)

In [ ]:
conesearch_records = conesearch_service.search(
    pos=conesearch_center,
    sr=conesearch_radius,
)
conesearch_records

## 4. Get the catalogue coverage

VizieR also provides the coverage of each catalogue with Multiple Order Coverage (<a href='https://ivoa.net/documents/MOC/'>MOC</a>) at order 10 -- meaning with a 0.001 rad spacial resolution.

In [ ]:
# retrieve the MOC
catalogue_coverage = MOC.from_vizier_table(CATALOGUE)
catalogue_coverage.display_preview()

The quick preview is the whole sky in equatorial coordinates. Red represent zones where the catalog has data, black areas are empty zones.

We can also plot the coverage with ``matplotlib``.

In [ ]:
fig = plt.figure(figsize=(5, 5))
wcs = catalogue_coverage.wcs(fig)
ax = fig.add_subplot(projection=wcs)
catalogue_coverage.fill(ax=ax, wcs=wcs, alpha=0.5, color="blue")

or with [ipyaladin](https://github.com/cds-astro/ipyaladin)

In [ ]:
aladin = Aladin()
aladin

We switch the target to the center of the conesearch

In [ ]:
aladin.target = "71.520996 +31.388166"

We can add the tables and coverage in this ipyaladin widget:

In [ ]:
aladin.add_table(conesearch_records.to_table(), color="lightskyblue", shape="plus")

If you don't see the points from the conesearch result, try zooming in the widget with your mouse wheel.

We can also add the result of the TAP query:

In [ ]:
aladin.add_table(tap_records.to_table())

In [ ]:
aladin.add_moc(
    catalogue_coverage,
    fill=True, opacity=0.5, color="pink", edge=True
)

For more information about manipulating multi-order coverages, see [MOCpy documentation](https://cds-astro.github.io/mocpy/), and about using the `ipyaladin` widget, see [its documentation](https://cds-astro.github.io/ipyaladin/).